In [1]:
import pandas as pd
import numpy as np
import csv
import os
import json
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import PunktSentenceTokenizer #ai that can be used to train on parts of speech
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
path ="metadata.csv" 
df = pd.read_csv(path, encoding = 'utf-8')

In [2]:
#directories = ["biorxiv_medrxiv","comm_use_subset","custom_license","noncomm_use_subset"]
directories = ["biorxiv_medrxiv"]
json_array = []
for d in directories:
    for file in os.listdir(f"{d}/{d}"):
        #print(file)
        file_path = f"{d}/{d}/{file}"
        j = json.load(open(file_path, "rb"))
        json_array.append(j)
#finds json files and saves to an array

In [3]:
len(json_array)

885

In [4]:
#search_keys 
for key in json_array[0]:
    print(key)

paper_id
metadata
abstract
body_text
bib_entries
ref_entries
back_matter


In [5]:
#start with 0 out of 885 lol I need to automate this but good for first test run
raw_abstract = []
raw_metadata = []
raw_bodytext = []
raw_bib_entries = []
raw_ref = []
raw_back_mater = []
for i in range(len(json_array)):
    raw_abstract.append(json_array[i]['abstract'])
    raw_metadata .append(json_array[i]['metadata'])
    raw_bodytext.append(json_array[i]['body_text'])
    raw_bib_entries.append(json_array[i]['bib_entries'])
    raw_ref.append(json_array[i]['ref_entries'])
    raw_back_mater.append(json_array[i]['back_matter'])

In [6]:
body_full_text = []
for i in range(len(raw_bodytext)):
    for key in raw_bodytext[i]:
        body_full_text.append(key['text'] + '\n\n')

In [7]:
##------------------------------------------------------------##
#raw_bodytext
#body_full_text = []
#for key in raw_bodytext:
    #body_full_text.append(key['text'] + '\n\n')
#    print(key['text'])

In [8]:
len(body_full_text)#there are 20 paragraphs here

30181

In [9]:
#needs more structuring.
#Tasks as per Kaggle:- what is known about transmission/incubation and environmental stability?
###Points to consider::--- Extract meaning(NLP?) Terms displayed in Tasks may be called something else in the dataset/research papers...(WordNet)
#body_full_text

In [10]:
#Range of incubation periods for the disease in humans (and how this varies across age and health status)
#and how long individuals are contagious, even after recovery.
#body_full_text[2]
#tokenize sentences

In [11]:
paragraphs = []
for i in range(len(body_full_text)):
    paragraphs.append(sent_tokenize(body_full_text[i]))

In [12]:
len(paragraphs[0]) #how many sentences are in a paragraph

2

In [13]:
#first step, remove stop words
len(paragraphs)

30181

## Stop words

In [14]:
stop_words = set(stopwords.words("english"))
print(stop_words)

{'above', 'been', 've', "you'd", 'there', 'shan', 'we', 'his', 'o', 'not', 'because', 'hadn', 'then', 'ain', 'during', "mightn't", "shan't", 'through', 'has', 'down', 'once', 'each', 'while', 'itself', 'under', 'herself', 'them', 'you', 'him', 'against', 'do', 'now', 'after', 'so', 'being', "she's", 'up', 'm', 'an', 'should', 'own', 'be', "hadn't", 'mustn', 'hasn', 'of', 'am', 'too', 'no', "needn't", 'she', 'as', 'in', 'here', "shouldn't", 'needn', 'than', 'such', 'why', 'yourself', 'will', "you'll", 'what', "couldn't", 'yours', 'he', 'doesn', "isn't", "didn't", "wasn't", 'this', 'had', 'having', 'into', 'was', 'ours', 'on', "should've", 'don', 'between', "hasn't", 'does', 't', 'mightn', 'weren', 'were', 'when', 'few', 'these', 'where', 'only', 'haven', 'is', 'how', 'd', 'didn', "aren't", 'about', "you're", 'but', 'theirs', 'below', 'can', 'me', 'yourselves', 'from', 'themselves', 'i', 'for', "doesn't", "it's", 'at', 'before', 'more', 'a', 'its', 'until', 'shouldn', 'whom', 'and', 'if'

In [15]:
i = 0
while i < len(paragraphs):
    temp = len(paragraphs[i])
    j = 0
    while j < temp:
        temp_word = paragraphs[i][j]
        temp_array = []
        temp_filtered = []
        temp_array = word_tokenize(temp_word)
        for word in temp_array:
            if word not in stop_words:
                temp_filtered.append(word)
        paragraphs[i][j] = temp_filtered
        j = j+ 1
    i = i+ 1
        

In [16]:
#paragraphs[1][3]

## Similarity Indexing

We are looking for range in incubation periods in humans only and how it varies across age and health status.
Step 1, generate a "similarity index" for each paragraph. The higher the similarity index, the more likely we are to find what we are looking for.

In [17]:
similarity_dictionary = {}

In [18]:
#target_words = ["incubation","humans","age","health"]
target_words = ["incubation","humans"]
target_length = len(target_words)
print (target_length)

2


In [19]:
len(paragraphs)

30181

Wordnet will be used to acquire these similarities. For my dictionary, i only need associated paragraph/sentence and attatched overall similarity in the form of a percentage. Start with base similarity search.

In [20]:
#data
#percentage
sentense_score_array = []
filtered_sentences = []
i = 0
while i < len(paragraphs):
    temp = len(paragraphs[i])
    j = 0
    while j < temp:
        temp_word = paragraphs[i][j]
        k = 0
        temp_score = 0
        raw_word_score = 0
        for word in temp_word:
            while k < target_length:
                try:
                    w1=wordnet.synsets(word)[0]
                    w2=wordnet.synsets(target_words[k])[0]
                    percentage = (w1.wup_similarity(w2))*100
                except:
                    percentage = 0
                percentage+=percentage
                k = k + 1
            raw_word_score += (percentage/target_length)
        try:
            raw_sentense_score = raw_word_score/len(temp_word)
        except:
            raw_sentense_score = 0
        #print("sentense_score: ",raw_sentense_score)
        sentense_score_array.append(raw_sentense_score)
        if(raw_sentense_score > 20):
            filtered_sentences.append(paragraphs[i][j])    
        j = j+ 1
    i = i+ 1  
print("Amount of filtered sentences", len(filtered_sentences))

Amount of filtered sentences 49966


In [21]:
#sentense_score_array
filtered_sentences[0]

['Due',
 'sequence',
 'similarity',
 'PKs',
 '(',
 'figure',
 '1C',
 ')',
 ',',
 'speculated',
 '88',
 'formed',
 'duplication',
 'events',
 'viral',
 'replication',
 ',',
 'probably',
 'involving',
 'recombination',
 '.']

In [22]:
#concatenate senteces for pattern recognition
i = 0
temp = ""
while i < len(filtered_sentences[0]):
    temp+=(filtered_sentences[0][i] + " ")
    i+=1

print(temp)

Due sequence similarity PKs ( figure 1C ) , speculated 88 formed duplication events viral replication , probably involving recombination . 


In [23]:
filtered_sentences[0] = temp

In [24]:
len(filtered_sentences)

49966

In [25]:
#concatenate for pattern matching
k = 0
while k < len(filtered_sentences):
    i = 0
    temp = ""
    while i < len(filtered_sentences[k]):
        temp+=(filtered_sentences[k][i] + " ")
        i+=1
    filtered_sentences[k] = temp
    #print(temp)
    k+=1

In [26]:
import statistics
print(statistics.mean(sentense_score_array))
max(sentense_score_array)

13.104143245669704


96.55172413793109

In [27]:
#import re
#remember..should be raw strings!
#x = "incubation is 523 days and 12 weeks"
#x = filtered_sentences[0]
#pattern = re.compile(r'\d+ (days|day|weeks|week|months|month|years|year)')
#matches = pattern.finditer(x)
#match_list = []
#for match in matches:
#    match_list.append(match)
#    print(match)
    
#print(x[5:7]) #string slicing

In [28]:
#remember..should be raw strings!
import re
i = 0
while i < len(filtered_sentences):
    pattern = re.compile(r'\d+ (days|day|weeks|week|months|month|years|year)')
    matches = pattern.finditer(filtered_sentences[i])
    match_list = []
    for match in matches:
        match_list.append(match)
        print(match)
    i+=1

<re.Match object; span=(62, 68), match='4 days'>
<re.Match object; span=(48, 54), match='2 days'>
<re.Match object; span=(31, 37), match='1 week'>
<re.Match object; span=(46, 52), match='5 days'>
<re.Match object; span=(141, 148), match='7 weeks'>
<re.Match object; span=(11, 17), match='8 days'>
<re.Match object; span=(28, 34), match='7 days'>
<re.Match object; span=(79, 88), match='18 months'>
<re.Match object; span=(233, 239), match='3 days'>
<re.Match object; span=(92, 99), match='10 days'>
<re.Match object; span=(27, 34), match='78 days'>
<re.Match object; span=(35, 43), match='241 days'>
<re.Match object; span=(70, 77), match='34 days'>
<re.Match object; span=(77, 85), match='80 years'>
<re.Match object; span=(64, 72), match='80 years'>
<re.Match object; span=(75, 84), match='160 years'>
<re.Match object; span=(17, 25), match='60 years'>
<re.Match object; span=(62, 69), match='20 days'>
<re.Match object; span=(130, 137), match='14 days'>
<re.Match object; span=(23, 30), match='35 

<re.Match object; span=(131, 137), match='2 days'>
<re.Match object; span=(27, 33), match='2 days'>
<re.Match object; span=(81, 87), match='4 days'>
<re.Match object; span=(102, 108), match='2 days'>
<re.Match object; span=(3, 11), match='20 month'>
<re.Match object; span=(127, 134), match='60 days'>
<re.Match object; span=(91, 98), match='14 days'>
<re.Match object; span=(75, 82), match='69 year'>
<re.Match object; span=(156, 163), match='59 year'>
<re.Match object; span=(62, 68), match='5 days'>
<re.Match object; span=(142, 148), match='2 days'>
<re.Match object; span=(72, 78), match='5 days'>
<re.Match object; span=(128, 134), match='1 days'>
<re.Match object; span=(61, 68), match='14 days'>
<re.Match object; span=(29, 35), match='6 days'>
<re.Match object; span=(80, 86), match='0 days'>
<re.Match object; span=(32, 38), match='3 days'>
<re.Match object; span=(96, 102), match='3 days'>
<re.Match object; span=(57, 64), match='24 days'>
<re.Match object; span=(36, 43), match='14 days'>

<re.Match object; span=(149, 156), match='14 days'>
<re.Match object; span=(302, 309), match='51 days'>
<re.Match object; span=(80, 87), match='12 days'>
<re.Match object; span=(109, 115), match='5 days'>
<re.Match object; span=(50, 57), match='65 week'>
<re.Match object; span=(213, 220), match='6 weeks'>
<re.Match object; span=(27, 34), match='14 days'>
<re.Match object; span=(69, 76), match='14 days'>
<re.Match object; span=(32, 39), match='13 days'>
<re.Match object; span=(78, 84), match='4 days'>
<re.Match object; span=(90, 96), match='7 days'>
<re.Match object; span=(78, 84), match='7 days'>
<re.Match object; span=(54, 61), match='2 years'>
<re.Match object; span=(43, 49), match='3 days'>
<re.Match object; span=(85, 92), match='14 days'>
<re.Match object; span=(140, 146), match='9 days'>
<re.Match object; span=(60, 68), match='18 years'>
<re.Match object; span=(56, 62), match='2 days'>
<re.Match object; span=(62, 68), match='1 days'>
<re.Match object; span=(132, 138), match='0 day

<re.Match object; span=(104, 112), match='21 weeks'>
<re.Match object; span=(40, 48), match='7 months'>
<re.Match object; span=(85, 93), match='9 months'>
<re.Match object; span=(109, 115), match='4 days'>
<re.Match object; span=(56, 62), match='8 days'>
<re.Match object; span=(105, 111), match='4 days'>
<re.Match object; span=(254, 261), match='77 days'>
<re.Match object; span=(280, 287), match='98 days'>
<re.Match object; span=(292, 299), match='29 days'>
<re.Match object; span=(34, 40), match='4 days'>
<re.Match object; span=(126, 132), match='8 days'>
<re.Match object; span=(51, 57), match='3 days'>
<re.Match object; span=(53, 62), match='18 months'>
<re.Match object; span=(125, 131), match='7 days'>
<re.Match object; span=(169, 175), match='0 days'>
<re.Match object; span=(229, 235), match='8 days'>
<re.Match object; span=(93, 99), match='2 days'>
<re.Match object; span=(129, 135), match='0 days'>
<re.Match object; span=(156, 163), match='14 days'>
<re.Match object; span=(25, 32),

<re.Match object; span=(42, 49), match='12 days'>
<re.Match object; span=(96, 103), match='2 weeks'>
<re.Match object; span=(16, 23), match='8 month'>
<re.Match object; span=(28, 34), match='2 days'>
<re.Match object; span=(10, 18), match='8 months'>
<re.Match object; span=(66, 75), match='16 months'>


In [29]:
match_list[0][0]

IndexError: list index out of range